In [1]:
import pandas as pd
import numpy as np
import uuid

In [3]:
# Read original data 
ray_data = pd.read_csv("../Original Data/ToFuTRESVER_14_7_27_2020_REV_22.csv")
locality_data = pd.read_csv("../Original Data/LOCAL_7_22_2020FuTRESVer12.csv")

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Fix SEX column 
female = ray_data['SEX']=="female"
male = ray_data['SEX'] == "male"
ray_data['SEX'][(female == False)&(male==False)]="not collected"

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
# Clean up Side column 
right = ray_data['SIDE']=="right"
left = ray_data['SIDE'] == "left"
ray_data['SIDE'][(female == False)&(male==False)]="not collected"

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
# Combine genus and species column 
ray_data = ray_data.assign(scientificName = ray_data['GENUS'] + " " + ray_data['SPECIES'])
ray_data["scientificName"] = ray_data["scientificName"].fillna("unknown")

In [7]:
# Create verbatimEventData column 
ray_data = ray_data.assign(verbatimEventDate = ray_data['DATE COLLECTED'])

# Create yearCollected column 
ray_data = ray_data.assign(yearCollected = "")

date_filter=ray_data["verbatimEventDate"].str.contains("Lower|Upper|Uppermost|Loc")
ray_data["yearCollected"]=ray_data["verbatimEventDate"][date_filter==False]

ray_data['yearCollected'] = ray_data.yearCollected.str[-4:]
ray_data['yearCollected'] = ray_data['yearCollected'].fillna("unknown")

In [8]:
# Create individualID column
ray_data = ray_data.assign(individualID = ray_data['SPEC_ID'])

In [9]:
# Create verbatimAgeValue columns 
ray_data = ray_data.assign(verbatimAgeValue = ray_data['AGE'])
ray_data = ray_data.assign(minimumChronometricAgeReferenceSystem ="mya")
ray_data = ray_data.assign(maximumChronometricAgeReferenceSystem ="mya")


# Create columns for minimumChronometricAge and maximumChronometricAge
ray_age_ranges = ray_data["verbatimAgeValue"].str.contains("-")
ray_ranges = ray_data["verbatimAgeValue"][ray_age_ranges==True]
ray_data = ray_data.assign(minimumChronometricAge = "")
ray_data = ray_data.assign(maximumChronometricAge = "")

for ind in ray_ranges.index:
    x = ray_ranges[ind]
    y = str(x)
    z = str(y).split("-")
    
    if z[0] > z[1]:
        ray_data['maximumChronometricAge'][ind] = z[0]
        ray_data['minimumChronometricAge'][ind] = z[1]
    else:
        ray_data['maximumChronometricAge'][ind] = z[1]
        ray_data['minimumChronometricAge'][ind] = z[0]
    
        
        

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_la

In [11]:
#Fix Country and Locality by matching to Locality sheet

ray_data = ray_data.assign(country = "")
ray_data = ray_data.assign(verbatimLocality = "")
ray_data = ray_data.assign(ray_temp_locality = "")

ray_data["COUNTRY"] = ray_data["COUNTRY"].apply(str)

ray_data['ray_temp_locality'] = ray_data['COUNTRY']+" "+ ray_data['LOCALITY']
locality_data['local_temp_locality'] = locality_data['COUNTRY No'] + " " + locality_data["LOCALITY No."]

for i in ray_data.index:
    for j in locality_data.index:
        if ray_data['ray_temp_locality'][i] == locality_data['local_temp_locality'][j]:
            ray_data["country"][i] = locality_data["COUNTRYName"][j]
            ray_data["verbatimLocality"][i] = locality_data["LOCALITYName"][j]
        else:
            ray_data["country"][i] = "Unknown"

            
ray_data['country']=ray_data['country'].replace({'Tibetan Plateau, Nepal':'Nepal'}) 

ray_data=ray_data.assign(locality= ray_data["verbatimLocality"]).fillna("Unknown")


/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [15]:
ray_data.to_csv('../Mapped Data/test.csv', index = False);

In [16]:
# Rearrange columns so that template columns are first, followed by measurement values

# Create column list
cols = ray_data.columns.tolist()

# Specify desired columns
cols = ['individualID',
        'SEX',
        'SIDE',
        'scientificName',
        'verbatimEventDate',
        'yearCollected',
        'verbatimAgeValue',
        'minimumChronometricAge',
        'maximumChronometricAge',
        'minimumChronometricAgeReferenceSystem',
        'maximumChronometricAgeReferenceSystem',
        'verbatimLocality',
        'locality',
        'country',
        'BONE',
        'M1',
        'M2',
        'M3',
        'M4',
        'M5',
        'M6',
        'M7',
        'M8',
        'M9',
        'M10',
        'M11',
        'M12',
        'M13',
        'M14',
        'M15',
        'M16',
        ' M17',
        'M18',
        'M19',
        'M20',
        'M21',
        'M22',
        'M23',
        'M24',
        'M25',
        'M26',
        'M27',
        'M28',
        'M29',
        'M30',
        'M31',
        'M32',
        'M33',
        'M34',
        'M35',
        'M36',
        'M37',
        'M38']

# Subset dataframe
ray_data = ray_data[cols]

In [17]:
# Matching template and column terms

# Renaming columns 
ray_data = ray_data.rename(columns = {'SEX':'sex',
                                      'SIDE':'side'})

In [18]:
# Create measurementUnit column
ray_data = ray_data.assign(measurementUnit="in")

ray_data["locality"]=ray_data["locality"].fillna("Unknown")

In [19]:
# Create basisOfRecord column
ray_data=ray_data.assign(basisOfRecord="")

recent_filter = ray_data['verbatimAgeValue']=="recent"
not_recent_filter = ray_data['verbatimAgeValue']!="recent"

ray_data['basisOfRecord'][recent_filter] = "PreservedSpecimen"
ray_data['basisOfRecord'][not_recent_filter] = "FossilSpecimen"


In [20]:
# Fill in blanks for required columns 
ray_data=ray_data.assign(samplingProtocol="Unknown")
ray_data=ray_data.assign(measurementMethod="Unknown")

In [21]:
# Create materialSampleID which is a UUID for each measurement
# Create eventID and populate it with materialSampleID

ray_data=ray_data.assign(materialSampleID = '')
ray_data['materialSampleID'] = [uuid.uuid4() for _ in range(len(ray_data.index))]

for ind in ray_data.index:
    x=ray_data['materialSampleID'][ind]
    y=str(x)
    z=y.replace("-", '_')
    
    ray_data['materialSampleID'][ind] = z

ray_data=ray_data.assign(eventID = ray_data["materialSampleID"])

In [22]:
#  Create long version so that each trait has its own row

# Creating long version, first specifiying keep variables, then naming variable and value
longVers=pd.melt(ray_data, 
                id_vars=['individualID',
                         'sex',
                         'side',
                         'scientificName',
                         'verbatimEventDate',
                         'yearCollected',
                         'verbatimAgeValue',
                         'minimumChronometricAge',
                         'maximumChronometricAge',
                         'minimumChronometricAgeReferenceSystem',
                         'maximumChronometricAgeReferenceSystem',
                         'verbatimLocality',
                         'locality',
                         'country',
                         'measurementUnit',
                         'basisOfRecord',
                         'samplingProtocol',
                         'measurementMethod',
                         'materialSampleID',
                         'eventID',
                         'BONE'], 
                var_name = 'measurementType', 
                value_name = 'measurementValue')
longVers

,individualID,sex,side,scientificName,verbatimEventDate,yearCollected,verbatimAgeValue,minimumChronometricAge,maximumChronometricAge,minimumChronometricAgeReferenceSystem,...,country,measurementUnit,basisOfRecord,samplingProtocol,measurementMethod,materialSampleID,eventID,BONE,measurementType,measurementValue
0,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,63b5adee_ea29_469d_9cab_886b7847b925,63b5adee_ea29_469d_9cab_886b7847b925,a1ph3,M1,77.2
1,SMNS7335,male,unknown,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,2e041512_fc20_4be4_8ad1_626baaa8f140,2e041512_fc20_4be4_8ad1_626baaa8f140,a2ph3,M1,39.4
2,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,573b7b56_f089_4d16_9783_960fc0b3d19b,573b7b56_f089_4d16_9783_960fc0b3d19b,astragalus,M1,55.7
3,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,79804438_2194_47c6_a854_b2fb836c2497,79804438_2194_47c6_a854_b2fb836c2497,calcaneum,M1,101.9
4,SMNS7335,male,unknown,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,fd4eb162_f825_4b3d_aea1_648598684c64,fd4eb162_f825_4b3d_aea1_648598684c64,femur,M1,357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807267,USNM541427,female,left,Equus hemionus,10-Jul-1981,1981,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,59a572c5_0b21_433a_8a83_b7393ea6a870,59a572c5_0b21_433a_8a83_b7393ea6a870,txP4,M38,Unknown
807268,MSNAF2842,not collected,not collected,"""Hipparion"" sp.",Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,18f1af18_132f_4161_b375_9135b1928555,18f1af18_132f_4161_b375_9135b1928555,txP3,M38,Unknown
807269,MSNAF2842,not collected,not collected,"""Hipparion"" sp.",Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,807aa04d_d32d_4e5a_a43c_567532104acd,807aa04d_d32d_4e5a_a43c_567532104acd,txP4,M38,Unknown
807270,USNM241009,male,left,Equus grevyi,9-Oct-1925,1925,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,f3c1e829_bfc7_4536_8542_97fee36c4199,f3c1e829_bfc7_4536_8542_97fee36c4199,tml1,M38,Unknown


In [23]:
# Create new measurementType column by combining BONE and measurementType column
longVers['measurementType'] = longVers['BONE'] + longVers['measurementType']
longVers['measurementType']

0              a1ph3M1
1              a2ph3M1
2         astragalusM1
3          calcaneumM1
4              femurM1
              ...     
807267         txP4M38
807268         txP3M38
807269         txP4M38
807270         tml1M38
807271      UnknownM38
Name: measurementType, Length: 807272, dtype: object

In [24]:
# Remove BONE columnn
del longVers['BONE']

In [25]:
# Filter out the GEOME measurements
correct_element_filter = longVers["measurementType"].str.match("femurM1|femurM2|humerusM1|humerusM2|astragalusM3|astraglausM2|astragalusM7|astragalusM1")
incorrect_filter = longVers["measurementType"].str.contains("M10|M11|M12|M13|M14|M15|M16|M17|M18|M19|M20|M21|M22|M23|M24|M25|M26|M27|M28|M29|M30|M31|M32|M33|M34|M35|M36|M37|M38")
longVers=longVers[correct_element_filter==True][incorrect_filter==False]

/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [26]:
# Renaming measurementType values

longVers['measurementType']=longVers['measurementType'].replace({'femurM1': 'femur length to greater trochanter', 
                                                                 'femurM2': 'femur length to head of femur', 
                                                                 'humerusM1': 'humerus length to ventral tubercle', 
                                                                 'humerusM2': 'humerus length to caput of humerus',
                                                                 'astragalusM3':'breadth of the trochlea of talus',
                                                                 #'astragalusM4':'talus breadth',
                                                                 'astraglausM2':'length of medial trochlea of talus',
                                                                 'astragalusM7':'depth of the medial side of talus',
                                                                 'astragalusM1':'length of the medial side of talus'
                                                                 #'astragalusM5':'distal articular breadth of talus',
                                                                 #'astragalusM6':'distal articular depth of talus'
                                                                 })


In [27]:
longVers['measurementType']

2         length of the medial side of talus
4         femur length to greater trochanter
5         humerus length to ventral tubercle
1504      length of the medial side of talus
1514      length of the medial side of talus
                         ...                
148416     depth of the medial side of talus
148417     depth of the medial side of talus
148418     depth of the medial side of talus
148519     depth of the medial side of talus
148520     depth of the medial side of talus
Name: measurementType, Length: 2802, dtype: object

In [38]:
# Create diagnosticID which is a unique number for each measurement
longVers=longVers.assign(diagnosticID = '')
longVers['diagnosticID'] = np.arange(len(longVers))

In [39]:
# Fix up measurementValue entries
#measurement_value_filter=longVers["measurementValue"].str.contains('frag|E|lat|NOT')
zero_values=longVers["measurementValue"]=="0"
blanks = longVers["measurementValue"] ==""
unknown = longVers["measurementValue"] == "Unknown"
locality_blanks = longVers["locality"]==""

longVers["measurementValue"]=longVers["measurementValue"][zero_values==False][blanks==False][unknown == False]
longVers["locality"][locality_blanks]="unknown"



/Users/neeka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [40]:
# If measurement value equals N/a, delete entire row
longVers = longVers.dropna(subset=['measurementValue'])

# Drop first row of data, it contains no measurementValue but is still retained
longVers = longVers.drop(longVers.index[0])

In [41]:
# Writing long data csv file
longVers.to_csv('../Mapped Data/FuTRES_Equid_Bernor_Global_Cenozoic.csv', index = False);

In [ ]:
#There's still an issue with measurementValue retaining blanks. I had to 
#go in and hand do it